In [ ]:
import sys
import subprocess

print("Python executable:", sys.executable)

subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "openpyxl"])


Python executable: C:\Users\josef\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


0

In [ ]:
import openai
from openai import OpenAI

print(openai.__version__)




2.8.1


In [ ]:
#import pandas as pd
#import matplotlib.pyplot as plt
#from vllm import LLM, SamplingParams
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="not-needed" 
)

MODEL_NAME = "Qwen2-0.5B-Instruct" 

In [ ]:
import pandas as pd

df = pd.read_excel("./testdata_MIE.xlsx")
df.to_csv("./testdata_MIE.csv", index=False)

df.head()


,PatientAccountID,MPINumber,AssessmentID,Source,CollectedDT,Finding,FindingName,FindingDataType,Value,AdditionalInformation
0,NaN,P-5xxxxxxxx,NaN,Stammdaten,Jul xx 20xx xx:xxAM,Geburtsdatum,NaN,9,xx/19xx,NaN
1,NaN,P-5xxxxxxxx,NaN,Stammdaten,Jun xx 20xx xx:xxPM,Geschlecht,NaN,1,männlich,NaN
2,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxAM,Diagnose,NaN,1,N18.0,Terminale Niereninsuffizienz
3,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxPM,Diagnose,NaN,1,G63.2*,Diabetische Polyneuropathie
4,P-8xxxxxxxxx,P-5xxxxxxxx,NaN,Diagnosen,Jun xx 20xx xx:xxPM,Diagnose,NaN,1,H36.0*,Retinopathia diabetica


In [ ]:
messages = [
    {"role": "user", "content": "Say hello from the Qwen server in one short sentence."}
]

response = client.chat.completions.create(
    model="Qwen/Qwen2-0.5B-Instruct",  
    messages=messages,
    temperature=0.1,
    max_tokens=100,
)

print(response.choices[0].message.content)




Hello from the Qwen server.


In [ ]:
from typing import List
import pandas as pd

def build_messages_for_query(df: pd.DataFrame, query: str) -> List[dict]:
    columns = list(df.columns)
    columns_str = ", ".join(columns)

    system_content = (
        "You write pure Python code to analyze an existing pandas DataFrame named df.\n"
        "df is already loaded in memory. You must:\n"
        "- NEVER load files (no pd.read_csv, no file paths).\n"
        "- NEVER overwrite df or assign a new object to df.\n"
        "- Use ONLY existing columns. The columns are: "
        f"{columns_str}\n"
        "- If a task depends on specific columns, first check them with:\n"
        "  required = {...}\n"
        "  missing = required.difference(df.columns)\n"
        "  if missing:\n"
        "      raise ValueError(f'Missing required columns: {missing}')\n"
        "\n"
        "Dataset semantics:\n"
        "- The dataset is event-based; patients are identified by 'MPINumber'.\n"
        "- Diagnoses: rows where Finding == 'Diagnose', diagnosis code in 'Value'.\n"
        "- Questions about codes like H36 or N18: filter Finding == 'Diagnose' and Value.str.contains(code), "
        "then count unique MPINumber.\n"
        "- Gender distribution: use column 'Gender' or 'Geschlecht' if present; otherwise raise ValueError. "
        "Create a pie chart with matplotlib.pyplot as plt and finish with plt.tight_layout() and plt.show().\n"
        "- Five most common secondary diagnoses: use rows with Finding == 'Diagnose', count Value frequencies, "
        "take top 5 and plot as bar chart.\n"
        "- HBA1C values: rows where Finding.str.contains('HBA1C', case=False, na=False) is True, value in 'Value'. "
        "For \"How many HBA1C values does each diagnosed patient have?\":\n"
        "  1) Find patients with at least one row Finding == 'Diagnose'.\n"
        "  2) Among those, select HBA1C rows.\n"
        "  3) Group by 'MPINumber' and count.\n"
        "\n"
        "Output format:\n"
        "- Output ONLY raw Python code, no comments, no explanations, no markdown, no backticks.\n"
        "- Do NOT leave bare expressions; always print results explicitly. "
        "For plots, import matplotlib.pyplot as plt and finish with plt.tight_layout() and plt.show().\n"
    )

    
    example_user = (
        "Example: How many patients got diagnosed with N18? "
        "Consider that each patient can have multiple findings."
    )

    example_assistant = (
        "required = {'Finding', 'Value', 'MPINumber'}\n"
        "missing = required.difference(df.columns)\n"
        "if missing:\n"
        "    raise ValueError(f'Missing required columns: {missing}')\n"
        "mask = (df['Finding'] == 'Diagnose') & df['Value'].str.contains('N18', na=False)\n"
        "subset = df.loc[mask]\n"
        "n_patients = subset['MPINumber'].nunique()\n"
        "print('Number of patients diagnosed with N18:', n_patients)"
    )

    real_user = (
        f"Task: {query}"
    )

    return [
        {"role": "system", "content": system_content},
        {"role": "user", "content": example_user},
        {"role": "assistant", "content": example_assistant},
        {"role": "user", "content": real_user},
    ]


def ask_llm(df: pd.DataFrame, query: str) -> str:
    """
    Schickt die Messages an den Qwen-Server und gibt den Code der Antwort zurück.
    Erwartung: reine Python-Code-Antwort, direkt mit exec() ausführbar.
    """
    messages = build_messages_for_query(df, query)

    response = client.chat.completions.create(
        model="Qwen/Qwen2-0.5B-Instruct",
        messages=messages,
        temperature=0.0,
        max_tokens=350, 
    )

    return response.choices[0].message.content



## RQ - A
How many people got diagnosed with H36 over time?

In [ ]:
query = "How many patients got diagnosed with H36? Consider that each patient can have multiple findings."
response_text = ask_llm(df, query)

print("Output:")
print(response_text)


Output:
required = {'Finding', 'Value', 'MPINumber'}
missing = required.difference(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')
mask = (df['Finding'] == 'Diagnose') & df['Value'].str.contains('H36', na=False)
subset = df.loc[mask]
n_patients = subset['MPINumber'].nunique()
print('Number of patients diagnosed with H36:', n_patients)


## RQ - B
How was the age and gender distribution of the disease?

In [ ]:
query = "Given the dataframe above, create a pie chart for the Gender distribution."

response_text = ask_llm(df, query)

print("Output:")
print(response_text)
print("#########")


Output:
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataframe
df = pd.read_csv('your_file.csv')

# Get the gender distribution
gender_distribution = df['Gender'].value_counts()

# Create a pie chart
plt.figure(figsize=(10, 6))
plt.pie(gender_distribution, labels=gender_distribution.index, autopct='%1.1f%%')
plt.title('Gender distribution')
plt.show()
#########


## RQ - C
What were the five most common secondary diagnoses?

In [ ]:
query = "Plot the five most common secondary diagnoses."

response_text = ask_llm(df, query)

print("Output:")
print(response_text)
print("#########")

try:
    exec(response_text)
except Exception as e:
    print("Error while executing LLM-generated code:", e)


Output:
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
df = pd.read_csv('your_file.csv')

# Get the required columns
required = {'Finding', 'Value', 'MPINumber'}
missing = required.difference(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')

# Filter the dataset
df = df[required]

# Count the number of patients with each diagnosis
df['Count'] = df['Finding'].str.contains('Diagnose', na=False).astype(int)
df['Count'].value_counts().plot(kind='bar', stacked=True, figsize=(10, 6))
plt.tight_layout()
plt.show()
#########
Error while executing LLM-generated code: [Errno 2] No such file or directory: 'your_file.csv'


## RQ - D
How many HBA1C values are there for each patient?

In [ ]:

query = "How many the HBA1C values has each diagnosed patient?"

response_text = ask_llm(df, query)

print("Output:")
print(response_text)
print("#########")

try:
    exec(response_text)
except Exception as e:
    print("Error while executing LLM-generated code:", e)

BadRequestError: Error code: 400 - {'object': 'error', 'message': "This model's maximum context length is 2048 tokens. However, you requested 2127 tokens (1327 in the messages, 800 in the completion). Please reduce the length of the messages or completion.", 'type': 'BadRequestError', 'param': None, 'code': 400}